In [0]:
# Change directory to VSCode workspace root so that relative path loads work correctly. Turn this addition off with the DataScience.changeDirOnImportExport setting
# ms-python.python added
import os
try:
	os.chdir(os.path.join(os.getcwd(), '..'))
	print(os.getcwd())
except:
	pass


In [1]:
import os
import numpy as np
import pandas as pd


In [2]:
os.chdir('notebook')


In [3]:
df = pd.read_csv('../output/notebook/2019-07-05_orthologs_for_maria.tsv', sep='\t', index_col=0)
df = df.reindex(['dmel', 'dpse', 'dmoj', 'dvir'], axis=1)


In [4]:
mappers = {
    'dmel': {
        'X': 'A',
        '2L': 'B',
        '2R': 'C', 
        '3L': 'D',
        '3R': 'E',
        '4': 'F',
        'Unmapped': np.nan,
        '211000022280328': np.nan
    }, 
    'dpse': {
        'XL': 'A',
        '4': 'B',
        '3': 'C', 
        'XR': 'D',
        '2': 'E',
        '5': 'F',
        'Unknown': np.nan
    }
}


In [5]:
def extract_and_map(x):
    _name = x.name
    mapper = mappers.get(_name, {'scaffold': np.nan})
    return (
        x.str.extract('([0-9a-zA-Z]+)', expand=False)
        .replace(mapper)
    )

muller = df.apply(extract_and_map, axis=0).dropna(how='all', axis=1)


In [6]:
muller_cnts = (
    muller.fillna('NAN')
    .groupby(['dpse', 'dmel']).size()
    .rename('count')
    .apply(lambda x: f'{x:,}')
    .to_frame()
    .reset_index()
)


In [7]:
muller['classD'] = np.nan
muller.loc[(muller.dmel == "D") & (muller.dpse == "D"), 'classD'] = 'ConservedD'
muller.loc[~((muller.dmel == "D") | muller.dmel.isna()) & (muller.dpse == "D"), 'classD'] = 'MovedOnD'
muller.loc[(muller.dmel == "D") & ~((muller.dpse == "D") | muller.dpse.isna()), 'classD'] = 'MovedOffD'
muller.loc[(muller.dmel == "D") & muller.dpse.isna(), 'classD'] = 'LostD'

muller['classE'] = np.nan
muller.loc[(muller.dmel == "E") & (muller.dpse == "E"), 'classE'] = 'ConservedE'
muller.loc[~((muller.dmel == "E") | muller.dmel.isna()) & (muller.dpse == "E"), 'classE'] = 'MovedOnE'
muller.loc[(muller.dmel == "E") & ~((muller.dpse == "E") | muller.dpse.isna()), 'classE'] = 'MovedOffE'
muller.loc[(muller.dmel == "E") & muller.dpse.isna(), 'classE'] = 'LostE'

muller.fillna('NAN').groupby(['classD', 'classE']).size().rename('counts').map(lambda x: f'{x:,}').to_frame()


counts
classD     classE           
ConservedD NAN         1,711
LostD      NAN           436
MovedOffD  MovedOnE       22
           NAN            25
MovedOnD   MovedOffE       3
           NAN             5
NAN        ConservedE  2,187
           LostE         533
           MovedOffE      20
           MovedOnE       19
           NAN         8,525

In [8]:
muller['groupD'] = np.nan
muller.loc[muller.classD.str.contains('Conserved') | muller.classD.str.contains('MovedOn'), 'groupD'] = 'SFD'
muller.loc[muller.classD.str.contains('Off').fillna(False), 'groupD'] = 'SAD'
muller.loc[muller.classD.str.contains('Lost').fillna(False), 'groupD'] = 'SAD?'

muller['groupE'] = np.nan
muller.loc[muller.classE.str.contains('Conserved') | muller.classE.str.contains('MovedOn'), 'groupE'] = 'SFE'
muller.loc[muller.classE.str.contains('Off').fillna(False), 'groupE'] = 'SAE'
muller.loc[muller.classE.str.contains('Lost').fillna(False), 'groupE'] = 'SAE?'

muller.fillna("NAN").groupby(['groupD', 'groupE']).size().rename('counts').map(lambda x: f'{x:,}').to_frame()


counts
groupD groupE       
NAN    NAN     8,525
       SAE        20
       SAE?      533
       SFE     2,206
SAD    NAN        25
       SFE        22
SAD?   NAN       436
SFD    NAN     1,716
       SAE         3